<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex08/patrick_ferreira/ex08_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelo de Linguagem com auto-atenção (versão eficiente)

Este exercício é similar ao da aula 5, mas iremos agora treinar *eficientemente* uma rede neural com uma ou mais camadas de auto-atenção para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Para tanto, deve-se implementar:
1. A máscara causal de atenção. Ela possibilitará que, durante o treinamento, com apenas uma forward+backward pass na rede, tenhamos as losses para todos os tokens de entrada (slide 117).
2. A máscara de PADs, que permite que usemos sequencias de comprimento variável no mesmo batch (slide 118).
3. Múltiplas cabeças.

## Importação dos pacotes

In [2]:
import collections
import itertools
import functools
import math
import os
import random
import re

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook, tqdm
from typing import List

In [3]:
# Check which GPU we are using
!nvidia-smi

Sat Oct 15 19:01:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    18W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if torch.cuda.is_available():
   dev = "cuda:0"
else:
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


# Carregamento do dataset 

Primeiro, fazemos download do dataset:

In [5]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



In [6]:
!pip install ptk-patrickctrf
!pip install transformers

from operator import itemgetter
from transformers import BertTokenizer
from ptk.utils import DataManager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [7]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
random.shuffle(x_train)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
x_train = x_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x in x_train[:3]:
    print(x[:100])

print('3 últimas amostras treino:')
for x in x_train[-3:]:
    print(x[:100])

print('3 primeiras amostras validação:')
for x in x_valid[:3]:
    print(x[:100])

print('3 últimas amostras validação:')
for x in x_valid[-3:]:
    print(x[:100])

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
If the myth regarding broken mirrors would be accurate, everybody involved in this production would 
Having read many of the comments here, I'm surprised that no one has recognized this as basically an
I really looked forward to seeing Nana after seeing Renoir amazing debut work, Whirlpool of Fate. I 
3 últimas amostras treino:
He only gets third billing (behind Arthur Treacher & Virginia Field), but this was effectively David
John Candy. Need we say more? He is the main reason you should see this film. Most people don't real
So I finally saw the film "My Left Foot" last night after years of being told by my mother how amazi
3 primeiras amostras validação:
ba ba ba boring...... this is next to battlefield earth in science fiction slumberness. genie franci
Please don't waste your time. This movie rehashes the worst of Bram Stoker's Dracula (Van Helsing), 
I really wanted to 

In [8]:
### Criando classe do dataset

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int, vocab_size=1000):
        self.context_size = context_size
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            print("Carregando dataset preprocessado")
        except Exception as e:
            # print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids
                for i in range(0, len(tokens_key)-context_size-1, context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+1:i+context_size+1])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long(), torch.tensor([False] * self.context_size)

Testando Dataset

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3, vocab_size=tokenizer.vocab_size)
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False)

assert len(dummy_dataset) == 3
print('Passou no assert de tamanho do dataset')

first_batch_input, first_batch_target, attention_mask = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[7327, 2175, 16033],
     [3449, 2050, 2175]])

correct_first_batch_target = torch.LongTensor(
    [[2175, 16033, 2139],
     [2050, 2175, 9153]])
assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Carregando dataset preprocessado
Passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


Dados de treino, validação e teste

In [10]:
# Load datasets
context_size = 15

# tokenizer = Tokenizador(x_train, tokenize, vocab_size=3000)
# tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

training_dataset = MyDataset(texts=x_train, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=x_valid, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=x_test, tokenizer=tokenizer, context_size=context_size)

print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

Montando dataset


100%|██████████| 20000/20000 [02:10<00:00, 153.60it/s]


Montando dataset


100%|██████████| 5000/5000 [00:28<00:00, 174.21it/s]


Montando dataset


100%|██████████| 25000/25000 [02:25<00:00, 171.44it/s]


training examples: 403792
valid examples: 100971
test examples: 492937


In [11]:
class MySequential(nn.Sequential):
    def forward(self, *input):
        for module in self._modules.values():
            input = module(*input)
        return input

class _AttentionLayer(torch.nn.Module):

    def __init__(self, embedding_dim: int, max_seq_length: int, ):
        """
        Implements the Self-attention, decoder-only."

        Args:
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super().__init__()
        self.context_size = max_seq_length
        self.embedding_dim = embedding_dim

        # Linear projections
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        # cast to probabilities
        self.softmax = nn.Softmax(dim=-1)

        # Matriz triangular de mascara, convertida para Booleano
        # Onde vale 1, o valor deve ser substituida por um valor negativo alto no tensor de scores.
        self.causal_mask = torch.ones((max_seq_length, max_seq_length), device=device).triu(diagonal=1) == 1.0

    def forward(self, x_embeddings, attention_mask):

        k = self.w_k(x_embeddings)
        v = self.w_v(x_embeddings)
        q = self.w_q(x_embeddings)

        scores = torch.matmul(q, k.transpose(1, 2))

        # Onde a mascara vale 1, retornamos um valor negativo grande.
        # Onde a mascara vale zero, mantemos intacto.
        probabilities = self.softmax(scores.masked_fill(self.causal_mask, -1e4).masked_fill(attention_mask.repeat_interleave(scores.shape[0] // attention_mask.shape[0],0).unsqueeze(-1), -1e4))

        return torch.matmul(probabilities, v)

class MultiHeadAttentionLayer(torch.nn.Module):

    def __init__(self, embedding_dim: int, max_seq_length: int, n_heads: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            n_heads (int): Number of self attention heads.
        """
        super().__init__()
        self.context_size = max_seq_length
        self.embedding_dim = embedding_dim
        self.head_dim = embedding_dim // n_heads
        self.n_heads = n_heads

        assert embedding_dim % n_heads == 0, "MultiHeadAttentionLayer Error: Embedding_dim must be an integer multiple of n_heads. "

        self.heads = _AttentionLayer(embedding_dim=self.head_dim, max_seq_length=max_seq_length)
        self.w_0 = nn.Linear(embedding_dim, embedding_dim)

        self.norm = nn.LayerNorm(embedding_dim)

    def forward(self, x_embeddings, attention_mask):

        return self.norm(
            x_embeddings +
            self.w_0(
                self.heads(
                    x_embeddings.reshape(x_embeddings.shape[0], x_embeddings.shape[1], self.n_heads, self.head_dim).movedim(1,2).reshape(-1, x_embeddings.shape[1], self.head_dim), attention_mask
                ).reshape(x_embeddings.shape[0], self.n_heads, x_embeddings.shape[1], self.head_dim).movedim(1,2).reshape(x_embeddings.shape[0], x_embeddings.shape[1], self.embedding_dim)
            )
        )


class DecoderLayer(torch.nn.Module):

    def __init__(self, embedding_dim: int, max_seq_length: int, n_heads: int, hidden_size: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            n_heads (int): Number of self attention heads.
            hidden_size (int): Number of neurons for feed-forward MLP.
        """
        super().__init__()
        self.context_size = max_seq_length
        self.embedding_dim = embedding_dim

        self.multihead_selfattention = MultiHeadAttentionLayer(embedding_dim, max_seq_length, n_heads)

        # output MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.LeakyReLU(),
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, embedding_dim)
        )

        self.norm = nn.LayerNorm(embedding_dim)

        # cast to probabilities
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x_embeddings, attention_mask):

        e = self.multihead_selfattention(x_embeddings, attention_mask)

        logits = self.mlp(e)

        return self.norm(logits + e), attention_mask

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, embedding_dim: int, n_layers: int, n_heads: int, hidden_size: int=2048):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            n_heads (int): Number of self attention heads.
            hidden_size (int): Number of neurons for feed-forward MLP.
        """
        super().__init__()
        context_size = max_seq_length
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # tokens (words indexes) embedding and positional embedding
        self.c_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.p_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_indexes = torch.arange(self.context_size, device=device).view(1, -1)

        self.decoder_layers = MySequential(*[DecoderLayer(embedding_dim=embedding_dim, max_seq_length=max_seq_length, n_heads=n_heads, hidden_size=hidden_size) for i in range(n_layers)])

        self.linear_output = nn.Linear(embedding_dim, vocab_size)

        # cast to probabilities
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, inputs, attention_mask):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)

        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        input_embeddings = self.c_embedding(inputs)

        positional_embeddings = self.p_embedding(self.positional_indexes.repeat(inputs.shape[0], 1))

        x_embeddings = positional_embeddings + input_embeddings

        logits, _ = self.decoder_layers(x_embeddings, attention_mask)

        return self.linear_output(logits)

Testando se o modelo processa os dados corretamente

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=context_size,
    embedding_dim=32,
    n_layers=1,
    n_heads=2,
    hidden_size=32,
).to(device)

sample_train, _, attention_mask = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
attention_mask = attention_mask.to(device)
model(sample_train_gpu, attention_mask).shape

torch.Size([1, 15, 30522])

Verificando a perplexidade

In [13]:
def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 100

sample_train, target_token_ids, attention_mask = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
attention_mask = attention_mask.to(device)
logits = model(sample_train_gpu, attention_mask)

print("vocab_size: ", tokenizer.vocab_size)

print("logits shape: ", logits.shape)

my_perplexity = perplexity(logits=logits, target=target_token_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=6000)
print('Passou o no assert da perplexidade')

del sample_train_gpu
del target_token_ids
del logits

vocab_size:  30522
logits shape:  torch.Size([100, 15, 30522])
my perplexity:              35078
correct initial perplexity: 30522
Passou o no assert da perplexidade


TREINAMENTO

In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast

max_examples = 20_000_000
eval_every_steps = 5000
lr = 3e-5
use_amp = True

unscaling_factor = 1

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=context_size,
    embedding_dim=512//unscaling_factor,
    n_layers=8//unscaling_factor,
    n_heads=8,
    hidden_size=2048//unscaling_factor,
).to(device)
train_loader = DataLoader(training_dataset, batch_size=1024//unscaling_factor, shuffle=True, num_workers=1)
validation_loader = DataLoader(valid_dataset, batch_size=1024//unscaling_factor, num_workers=1, )

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, min_lr=8e-6, patience=3, threshold=5e-3, verbose=True)
scaler=GradScaler()


def train_step(input_ids, target_ids, attention_mask):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        logits = model(input_ids, attention_mask)
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, )
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(input_ids, target_ids, attention_mask):
    model.eval()
    with torch.no_grad():
        with autocast(enabled=use_amp):
            logits = model(input_ids, attention_mask)
            logits = logits.reshape(-1, logits.shape[-1])
            target_ids = target_ids.reshape(-1)
            loss = nn.functional.cross_entropy(logits, target_ids,)
    return loss.item()

best_validation_ppl = 9999
train_losses = []
n_examples = 0
step = 0
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for train_input_ids, train_target_ids, attention_mask in DataManager(train_loader, device=device, buffer_size=1, data_type=None):
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device), attention_mask)
        train_losses.append(loss)

        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            # LR scheduler
            scheduler.step(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device), attention_mask)
                    for val_input_ids, val_target_ids, attention_mask in DataManager(validation_loader, device=device, buffer_size=1, data_type=None)]))
                # Checkpoint to best models found.
                if best_validation_ppl > valid_ppl:
                    # Update the new best perplexity.
                    best_validation_ppl = valid_ppl
                    model.eval()
                    torch.save(model, "best_model.pth")

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        pbar.update(len(train_input_ids))
        if n_examples >= max_examples:
            break

pbar.close()

# Restore best model (checkpoint) found
model = torch.load("best_model.pth")
model.eval()

  0%|          | 1024/20000000 [00:16<86:58:08, 63.88it/s]

0 steps; 0 examples so far; train ppl: 36032.89, valid ppl: 35609.71


 26%|██▌       | 5112768/20000000 [37:41<22:55:21, 180.40it/s]

5000 steps; 5111744 examples so far; train ppl: 173.08, valid ppl: 109.39


 51%|█████     | 10223824/20000000 [1:15:03<14:59:52, 181.07it/s]

10000 steps; 10222800 examples so far; train ppl: 84.69, valid ppl: 90.00


 77%|███████▋  | 15335568/20000000 [1:52:23<7:14:11, 179.05it/s]

15000 steps; 15334544 examples so far; train ppl: 65.37, valid ppl: 85.00


20000848it [2:26:08, 2281.05it/s]


LanguageModel(
  (c_embedding): Embedding(30522, 512)
  (p_embedding): Embedding(30522, 512)
  (decoder_layers): MySequential(
    (0): DecoderLayer(
      (mlp): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (3): Dropout(p=0.1, inplace=False)
        (4): Linear(in_features=2048, out_features=512, bias=True)
      )
      (multihead_selfattention): MultiHeadAttentionLayer(
        (heads): _AttentionLayer(
          (w_q): Linear(in_features=64, out_features=64, bias=True)
          (w_k): Linear(in_features=64, out_features=64, bias=True)
          (w_v): Linear(in_features=64, out_features=64, bias=True)
          (softmax): Softmax(dim=-1)
        )
        (w_0): Linear(in_features=512, out_features=512, bias=True)
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (softmax): Softmax(dim=-1)
      

Avaliação no dataset de Teste

In [17]:
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=1)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device), attention_mask.to(device))
        for input, target, attention_mask in tqdm(test_loader)
    ]))

print(f'test perplexity: {test_ppl}')



  0%|          | 0/7703 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa47a828440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__


    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
  0%|          | 2/7703 [00:00<07:33, 16.98it/s]    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
  0%|          | 0/7703 [23:34<?, ?it/s]


  0%|          | 4/7703 [00:00<07:14, 17.71it/s]

  0%|          | 8/7703 [00:00<04:49, 26.56it/s]

  0%|          | 13/7703 [00:00<03:53, 32.98it/s]

  0%|          | 18/7703 [00:00<03:27, 37.06it/s]

  0%|          | 23/7703 [00:00<03:08, 40.69it/s]

  0%|          | 28/770

test perplexity: 92.14669598166776


Testando gerar uma sentença

In [25]:
prompt = 'Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to '
max_output_tokens = 30

for _ in range(max_output_tokens):
    input_ids = tokenizer(prompt, return_tensors=None, add_special_tokens=False).input_ids
    input_ids_truncated = input_ids[-context_size:]# Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device), torch.tensor([[False] * context_size], device=device))[:, -1, :]
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    predicted_word = tokenizer.decode([predicted_id,])
    prompt += predicted_word[0]
    print(prompt)

Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b.
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b.c
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b.c.
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b.c.,
Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to t.<b.c.,b
Frankenstein tells the story of gifted scienti